# Delta Lake Deep Dive – Storage, ACID, and Optimization

This document explains **how Delta Lake works internally**, how data is stored and versioned, and how performance optimizations such as **predicate pushdown, OPTIMIZE, ZORDER, and VACUUM** affect physical data layout.

The walkthrough uses a **sales dataset** and demonstrates reading, writing, updating, optimizing, and organizing data into the appropriate lakehouse layer.


## 1. Sales Dataset

The dataset represents transactional sales data generated by an online retail system. Each record represents a single order line item.

**Columns**
- customer_id: Unique customer identifier
- order_id: Order identifier
- country: Customer country
- product: Product category
- quantity: Units sold
- amount: Total order amount


In [0]:
%sql
CREATE CATALOG IF NOT EXISTS training_catalog;


In [0]:
%sql
USE CATALOG training_catalog;

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS training_catalog.default.bronze

In [0]:
sales_data = [
    (1, "ORD-001", "India", "Laptop", 1, 75000),
    (2, "ORD-002", "USA", "Phone", 2, 40000),
    (3, "ORD-003", "India", "Tablet", 1, 30000),
    (4, "ORD-004", "UK", "Laptop", 1, 72000),
    (5, "ORD-005", "India", "Phone", 3, 60000)
]

columns = ["customer_id", "order_id", "country", "product", "quantity", "amount"]
df_sales = spark.createDataFrame(sales_data, columns)
df_sales.show()

+-----------+--------+-------+-------+--------+------+
|customer_id|order_id|country|product|quantity|amount|
+-----------+--------+-------+-------+--------+------+
|          1| ORD-001|  India| Laptop|       1| 75000|
|          2| ORD-002|    USA|  Phone|       2| 40000|
|          3| ORD-003|  India| Tablet|       1| 30000|
|          4| ORD-004|     UK| Laptop|       1| 72000|
|          5| ORD-005|  India|  Phone|       3| 60000|
+-----------+--------+-------+-------+--------+------+



## 2. Writing Data as a Delta Table (Bronze Layer)

When data is written using the Delta format, two things happen:
1. Data is stored as Parquet files
2. A transaction log is created to track table state


In [0]:
df_sales.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("sales_delta")

## 3. Delta Transaction Log

Delta Lake maintains a directory named `_delta_log` at the root of every Delta table. This directory contains **ordered commit files** that define the table state.

### JSON Commit Files
Each JSON file represents a **single atomic transaction**. The file records:
- Files added to the table
- Files removed from the table
- Schema and metadata changes

Delta Lake uses **optimistic concurrency control**, ensuring that concurrent writers do not corrupt data.
Delta Transaction Log (How ACID Works)

Delta Lake records **every change** in `_delta_log`.
- JSON files → individual commits
- Parquet checkpoint → optimized snapshots

In [0]:
%sql
DESCRIBE DETAIL sales_delta;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,916a4977-8d33-4380-802f-fbff1fa60ff2,training_catalog.default.sales_delta,null,,2026-01-19T09:38:31.497Z,2026-01-19T10:15:04.000Z,List(),List(),1,2029,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
%sql
DESCRIBE HISTORY sales_delta;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
6,2026-01-19T10:15:04.000Z,78469401222449,pavan@learnlytica.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1155444108437728),0119-084634-kzuiloq-v2n,5,SnapshotIsolation,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 2044, p25FileSize -> 2029, numDeletionVectorsRemoved -> 1, minFileSize -> 2029, numAddedFiles -> 1, maxFileSize -> 2029, p75FileSize -> 2029, p50FileSize -> 2029, numAddedBytes -> 2029)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
5,2026-01-19T10:15:02.000Z,78469401222449,pavan@learnlytica.com,DELETE,"Map(predicate -> [""(order_id#16534 = ORD-002)""])",null,List(1155444108437728),0119-084634-kzuiloq-v2n,4,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1538, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 1005, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 531)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
4,2026-01-19T10:14:08.000Z,78469401222449,pavan@learnlytica.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1155444108437728),0119-084634-kzuiloq-v2n,3,SnapshotIsolation,false,"Map(numRemovedFiles -> 4, numRemovedBytes -> 7142, p25FileSize -> 2044, numDeletionVectorsRemoved -> 1, minFileSize -> 2044, numAddedFiles -> 1, maxFileSize -> 2044, p75FileSize -> 2044, p50FileSize -> 2044, numAddedBytes -> 2044)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
3,2026-01-19T10:14:06.000Z,78469401222449,pavan@learnlytica.com,UPDATE,"Map(predicate -> [""(country#16017 = India)""])",null,List(1155444108437728),0119-084634-kzuiloq-v2n,2,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3167, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1555, numAddedFiles -> 1, numUpdatedRows -> 3, numAddedBytes -> 1863, rewriteTimeMs -> 1558)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
2,2026-01-19T10:13:46.000Z,78469401222449,pavan@learnlytica.com,WRITE,"Map(mode -> Append, statsOnLoad -> true, partitionBy -> [])",null,List(1155444108437728),0119-084634-kzuiloq-v2n,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1675)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2026-01-19T10:00:42.000Z,78469401222449,pavan@learnlytica.com,WRITE,"Map(mode -> Append, statsOnLoad -> true, partitionBy -> [])",null,List(1155444108437728),0119-084634-kzuiloq-v2n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1675)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2026-01-19T09:38:34.000Z,78469401222449,pavan@learnlytica.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(1155444108437728),0119-084634-kzuiloq-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 5, numOutputBytes -> 1929)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


In [0]:
%fs
ls /Volumes/training_catalog/default/sales_delta/_delta_log

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-6029603466042821>, line 1
----> 1 display(dbutils.fs.ls("/Volumes/training_catalog/default/sales_delta/_delta_log"))

File /databricks/python_shell/lib/dbruntime/remotefshandler/RemoteFsHandler.py:52, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     49 class ExecutionError(Exception):
     50     pass
---> 52 raise ExecutionError(str(e)) from None

ExecutionError: [UC_VOLUME_NOT_FOUND] Volume `training_catalog`.`default`.`sales_delta` does not exist. Please use 'SHOW VOLUMES' to list available volumes. SQLSTATE: 42704

JVM stacktrace:
org.apache.spark.sql.catalyst.analysis.NoSuchVolumeException
	at com.databricks.sql.managedcatalog.client.ManagedCatalogClientImpl$ConvertVolumeException.convertVolumeException(ManagedCatalogClientImpl.scala:9233)
	at com.databricks.sql.managedcatalog.cli

### ACID Guarantees

- **Atomicity**: Transactions either fully commit or fail
- **Consistency**: Schema and constraints are enforced
- **Isolation**: Readers see a consistent snapshot
- **Durability**: Changes are persisted in the transaction log


## 4. Data Modifications (INSERT, UPDATE, DELETE)

Delta Lake never modifies Parquet files in place. Instead, it writes new files and updates the transaction log to reflect changes.

In [0]:
%sql
INSERT INTO sales_delta VALUES
(6, 'ORD-006', 'USA', 'Laptop', 1, 80000)

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE sales_delta
SET amount = amount + 5000
WHERE country = 'India'

num_affected_rows
3


In [0]:
%sql
DELETE FROM sales_delta
WHERE order_id = 'ORD-002'

num_affected_rows
1


## 5. Time Travel

Delta Lake reconstructs table state by replaying the transaction log. This allows querying historical versions of the data.
(Audit & Debugging)

**Use cases:**
- Debug broken pipelines
- Re-run ML training
- Regulatory audits

In [0]:
%sql
DESCRIBE HISTORY sales_delta

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6029603466042828>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'DESCRIBE HISTORY sales_delta\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:152, in SqlMagic.sql(self, line, cell)
    148     raise Exception(
    149         "Cannot run %sql command because spark 

## 6. Predicate Pushdown and Data Skipping

Predicate pushdown allows query filters to be applied at the storage layer. Delta Lake stores column-level statistics (min/max) in the transaction log.

During query execution, files that cannot satisfy filter conditions are skipped entirely, reducing I/O.


In [0]:
%sql
SELECT * FROM sales_delta;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8038335162689490>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'SELECT * FROM sales_delta;\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:152, in SqlMagic.sql(self, line, cell)
    148     raise Exception(
    149         "Cannot run %sql command because spark co

## 7. Small Files Problem

Frequent writes generate many small Parquet files. Each file introduces metadata and I/O overhead, which degrades query performance over time.

In [0]:
df_sales.repartition(20) \
  .write \
  .mode("overwrite") \
  .saveAsTable("sales_small_files1")

## 8. OPTIMIZE

OPTIMIZE rewrites many small files into fewer, larger files. The transaction log is updated to reference the new files while preserving historical versions.
**What OPTIMIZE does:**
- Compacts small files
- Improves scan performance
- Reduces metadata overhead

In [0]:
%sql
SELECT * FROM sales_small_files1 where product='Laptop';

customer_id,order_id,country,product,quantity,amount
1,ORD-001,India,Laptop,1,75000
4,ORD-004,UK,Laptop,1,72000


In [0]:
%sql
SELECT * FROM sales_small_files where product='Laptop';

customer_id,order_id,country,product,quantity,amount
4,ORD-004,UK,Laptop,1,72000
1,ORD-001,India,Laptop,1,75000


In [0]:
%sql
OPTIMIZE sales_small_files

path,metrics
,"List(1, 4, List(1930, 1930, 1930.0, 1, 1930), List(1707, 1832, 1741.0, 4, 6964), 0, null, null, 0, 1, 4, 0, true, 0, 0, 1768821611665, 1768821615763, 8, 1, null, List(0, 0), null, 6, 6, 813, 0, null)"


## 9. Z-ORDER

Z-Ordering reorganizes data so that related values are stored close together. This improves data skipping when filters are applied on Z-ordered columns.

**What ZORDER does:**
- Physically co-locates related data
- Enables efficient predicate pruning
- Reduces file scans

In [0]:
%sql
OPTIMIZE sales_small_files
ZORDER BY (country, product)

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1930), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1768822254542, 1768822255205, 8, 0, null, List(0, 0), null, 6, 6, 0, 0, null)"


## 10. VACUUM

VACUUM removes unreferenced data files older than the retention period. This frees storage but limits time travel capability.

In [0]:
**What VACUUM does:**
- Removes obsolete data files
- Frees storage
- Limits time travel

⚠️ Never reduce retention blindly in production.


In [0]:
%sql
VACUUM sales_small_files RETAIN 168 HOURS

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## 11. Writing Data to the Silver Layer

The Silver layer contains cleansed, optimized, and query-efficient data.

In [0]:
%sql
CREATE TABLE training_catalog.silver.sales_silver
USING DELTA
AS SELECT * FROM training_catalog.bronze.sales_small_files